## Environments

In [1]:
!python -V

Python 3.7.12


In [2]:
import torch, torchvision
torch.__version__, torchvision.__version__

('1.6.0+cu101', '0.7.0+cu101')

In [5]:
%cd mmrotate

/home/ubuntu/src/article-009/src/mmrotate


In [6]:
!rm -r /workspace/src/mmrotate/mmrotate.egg-info
!pip install -r requirements/build.txt
!pip install -v -e .

rm: cannot remove '/workspace/src/mmrotate/mmrotate.egg-info': No such file or directory
  Using cached Cython-0.29.35-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
Using pip 23.1.2 from /home/ubuntu/.conda/envs/article-009/lib/python3.7/site-packages/pip (python 3.7)
Obtaining file:///home/ubuntu/src/article-009/src/mmrotate
  Preparing metadata (setup.py) ...   Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-gro8jv5k/mmrotate.egg-info
  writing /tmp/pip-pip-egg-info-gro8jv5k/mmrotate.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-gro8jv5k/mmrotate.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-gro8jv5k/mmrotate.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-gro8jv5k/mmrotate.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-gro8jv5k/mmrotate.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pi

In [8]:
# notebook directory
%cd ../

/home/ubuntu/src/article-009/src


In [1]:
import mmcv
import mmrotate ## mmcv>=1.3.17, <=1.8.0.
# mmcv.__version__, mmrotate.__version__

/home/ubuntu/.conda/envs/article-009/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## GPU Check

In [2]:
import torch 
# gpu check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.cuda.is_available()

cuda


True

In [3]:
!nvidia-smi

Thu Jul  6 09:11:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.08    Driver Version: 510.73.08    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   52C    P0    69W /  70W |   8955MiB / 15360MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Pretrained model

In [11]:
!apt install wget

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  wget
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 316 kB of archives.
After this operation, 954 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 wget amd64 1.19.4-1ubuntu2.2 [316 kB]
Fetched 316 kB in 1s (219 kB/s)3m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package wget.
(Reading database ... 18025 files and directories currently installed.)
Preparing to unpack .../wget_1.19.4-1ubuntu2.2_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 17%] [#########.................................................] 8Unpacking wget (1.19.4-1ubuntu2.2) ...
7Progress: [ 33%] [###################.......................................] 87Progre

In [13]:
!wget https://download.openmmlab.com/mmrotate/v0.1.0/orientedreppoints/oriented_reppoints_r50_fpn_1x_dota_le135/oriented_reppoints_r50_fpn_1x_dota_le135-ef072de9.pth  -P ./weights/

--2023-07-04 04:08:05--  https://download.openmmlab.com/mmrotate/v0.1.0/orientedreppoints/oriented_reppoints_r50_fpn_1x_dota_le135/oriented_reppoints_r50_fpn_1x_dota_le135-ef072de9.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 128.1.157.210, 128.1.157.217, 128.1.157.212, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|128.1.157.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147620842 (141M) [application/octet-stream]
Saving to: ‘./weights/oriented_reppoints_r50_fpn_1x_dota_le135-ef072de9.pth’

oriented_reppoints_ 100%[===================>] 140.78M  31.3MB/s    in 4.6s    

2023-07-04 04:08:09 (30.9 MB/s) - ‘./weights/oriented_reppoints_r50_fpn_1x_dota_le135-ef072de9.pth’ saved [147620842/147620842]



## DOTA v1

In [11]:
from pycocotools.coco import COCO
from tqdm import tqdm
import os
import cv2

COCO_ROOT = '../sample/train_scale1_h640_w640_oh0.5_ow0.5_min1_segs'

In [16]:
for phase in ['train', 'valid']:
    
    COUNT_NO_LABEL = 0
    
    os.makedirs(COCO_ROOT + '/' + phase, exist_ok=True)
    coco = COCO(f'{COCO_ROOT}/{phase}.json')

    catIds = coco.getCatIds(catNms=['car']);
    imgIds = coco.getImgIds(catIds=catIds);
    
    print('\n' , f'{len(imgIds)} images for {phase}')
    
    for idx_img in tqdm(imgIds, total=len(imgIds)):
        try:
            img_info = coco.loadImgs(imgIds[idx_img])[0]
            annIds = coco.getAnnIds(imgIds=img_info['id'], catIds=catIds, iscrowd=None)
            anns = coco.loadAnns(annIds)
            
            DOTA_FMTS = []
            
            for ann in anns:
                bbox = ann['segmentation'][0]
                if len(bbox) < 8:
                    # put tmp point
                    bbox += bbox
                
                bbox = bbox[:8]
                bbox = list(map(str,map(round, bbox)))
                DOTA_FMTS.append(' '.join(bbox) + f' car 0')
                
            img = cv2.imread(f'{COCO_ROOT}/{img_info["file_name"]}')
            cv2.imwrite(f'{COCO_ROOT}/{phase}/{img_info["file_name"]}', img)
                
            # write to txet file
            with open(f'{COCO_ROOT}/{phase}/{img_info["file_name"]}.txt', 'w') as f:
                f.write('\n'.join(DOTA_FMTS))
        except Exception as e:
            COUNT_NO_LABEL += 1
            
    print(f'{COUNT_NO_LABEL} images have no label phase {phase}')
            

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!

 392 images for train


100%|██████████| 392/392 [00:05<00:00, 72.11it/s]


9 images have no label phase train
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

 215 images for valid


100%|██████████| 215/215 [00:00<00:00, 409.53it/s]

178 images have no label phase valid


In [9]:
!cp dota.py mmrotate/mmrotate/datasets/dota.py

## Train

In [1]:
!python mmrotate/tools/train.py cfg/oriented_reppoints_segs.py

/home/ubuntu/src/article-009/src/mmrotate/mmrotate/utils/setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/home/ubuntu/src/article-009/src/mmrotate/mmrotate/utils/setup_env.py:49: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment variable for each process '
2023-07-07 04:07:26,860 - mmrotate - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:21) [GCC 9.4.0]
CUDA available: True
GPU 0: Tesla T4
CUDA